In [30]:
import pandas as pd

In [31]:
df=pd.read_csv('../data/nyc_311_request_data_dec_2025.csv')

In [32]:
drop_columns=['taxi_company_borough','facility_type','road_ramp','location','bridge_highway_direction','due_date','bridge_highway_segment','bridge_highway_name','taxi_pick_up_location','vehicle_type','descriptor_2','landmark','intersection_street_1','intersection_street_2','cross_street_1','cross_street_2','bbl','street_name','x_coordinate_state_plane','y_coordinate_state_plane','park_borough','park_facility_name','agency_name']
len(set(drop_columns))

23

In [33]:
df.drop(columns=drop_columns,inplace=True)

In [34]:
null_record_drop_columns=['incident_zip','longitude','latitude','council_district','incident_address','city']
len(set(null_record_drop_columns))

6

In [35]:
df.dropna(subset=null_record_drop_columns,how='any',inplace=True)

In [36]:
Unpecified_columns=['location_type','resolution_description','descriptor','address_type',]
len(set(Unpecified_columns))

4

In [37]:
df[Unpecified_columns]=df[Unpecified_columns].fillna("unspecified")

In [38]:
datetime_columns=['created_date','closed_date','resolution_action_updated_date']
len(set(datetime_columns))

3

In [39]:
for col in datetime_columns:
    df[col]=pd.to_datetime(df[col],errors='coerce')

In [40]:
df['council_district'] = (
    df['council_district']
    .astype('Int64')
)

In [41]:
categorical_colums=['location_type','address_type','status','city','council_district','community_board','police_precinct','borough','agency','complaint_type','open_data_channel_type','descriptor']
len(set(categorical_colums))

12

In [42]:
for col in categorical_colums:
    df[col]=df[col].astype('category')

In [43]:
df['incident_zip'] = (
    df['incident_zip']
    .astype('Int64')
    .astype(str)
    .str.zfill(5)
)

##### Invalid data check

In [44]:
duplicates_mask = df.duplicated()

num_duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {num_duplicates}")

df_unique = df[~duplicates_mask]

df_unique = df.drop_duplicates()


Number of duplicate rows: 0


In [45]:
(df['closed_date']-df['created_date'] < pd.Timedelta(0)).value_counts()

False    305003
True         16
Name: count, dtype: int64

In [46]:
((~df['closed_date'].isna()) & (df['status']!='Closed')).value_counts()

False    302336
True       2683
Name: count, dtype: int64

##### 1. The 16 requests which are closed even before raised should not exist
##### 2. The 2683 requests which are closed but still not marked as Closed in status should not exist.

Action : Drop invalid rows


In [47]:
invalid_data_mask= (df['closed_date']-df['created_date'] < pd.Timedelta(0)) & ((~df['closed_date'].isna()) & (df['status']!='Closed'))
df=df[~invalid_data_mask]

In [48]:
# df.to_csv('../data/cleaned_data_dec_2025.csv')

In [49]:
import sys
import subprocess

if 'pyarrow' not in sys.modules:
    try:
        import pyarrow
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "pyarrow"])
        import pyarrow

print("pyarrow is ready to use!")


pyarrow is ready to use!


In [ ]:
# df_unique.to_parquet('../data/cleaned_data_dec_2025.parquet', engine='pyarrow', index=False)

In [52]:
df.info()

<class 'pandas.DataFrame'>
Index: 305003 entries, 0 to 323342
Data columns (total 21 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   unique_key                      305003 non-null  int64         
 1   created_date                    305003 non-null  datetime64[us]
 2   closed_date                     286553 non-null  datetime64[us]
 3   agency                          305003 non-null  category      
 4   complaint_type                  305003 non-null  category      
 5   descriptor                      305003 non-null  category      
 6   location_type                   305003 non-null  category      
 7   incident_zip                    305003 non-null  str           
 8   incident_address                305003 non-null  str           
 9   address_type                    305003 non-null  category      
 10  city                            305003 non-null  category      
 11  sta